### HOJA DE TRABAJO 2

KENNETH GALVEZ 20079

JESSICA ORTIZ 20192

ESTEBAN ALDANA 20591

In [19]:
# Importando bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


### Task 1 - Preparación del conjunto de datos

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos
en conjuntos de entrenamiento y validación.


In [20]:
# Cargando el conjunto de datos de Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividiendo el conjunto de datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Conviértelos a tensores de PyTorch
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)


### Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas
ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [21]:
# Definir la arquitectura del modelo
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size, regularization):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, output_size)
        self.dropout = nn.Dropout(0.2) if regularization == 'Dropout' else None

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        if self.dropout is not None:
            x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.fc3(x)
        return x

In [22]:
# Configuración de hiperparámetros
input_size = X_train.shape[1]
output_size = len(torch.unique(y_train))
num_epochs = 50
learning_rate = 0.01

best_accuracy = 0
best_combo = None

### Task 3-4-5 - Funciones de Pérdida - Tecnicas de REgularizacion - Algoritmos de optimizacion



In [23]:
# Possible loss functions
loss_functions = ['CrossEntropy', 'MSE', 'Poisson']

#### Task 6 - Experimentación y Análisis
Entrene los modelos con diferentes combinaciones de funciones de pérdida, técnicas de regularización y
algoritmos de optimización. Para no complicar esta parte, puede dejar fijo dos de estos parámetros (función de
pérdida, técnicas de regularización, algoritmo de optimización) y solamente cambiar uno de ellos. Deben verse al
menos 9 combinaciones en total, donde es válido que en una de ellas no haya ninguna técnica de regularización. Si
quiere experimentar con más combinaciones se le dará hasta 10% de puntos extra.
Para cada combinación registre métricas como precisión, pérdida y alguna otra métrica que considere pertinente
(Recuerde lo visto en inteligencia artificial).
Visualice las curvas (tanto en precisión, pérdida y la tercera métrica que decidió) de entrenamiento y validación
utilizando bibliotecas como matplotlib y/o seaborn. Además, recuerde llevar tracking de los tiempos de ejecución
de cada combinación

In [26]:
# Realizar todas las combinaciones posibles
for loss_function_name in loss_functions:
    for regularization in ['L1', 'L2', 'Dropout']:
        for optimizer_name in ['SGD', 'Adam', 'RMSprop']:
            
            model = SimpleNN(input_size, output_size, regularization=regularization)

            # Establecer la función de pérdida
            loss_function = nn.CrossEntropyLoss()

            # Establecer el optimizador
            if optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=(1e-4 if regularization == 'L2' else 0))
            elif optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=(1e-4 if regularization == 'L2' else 0))
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=(1e-4 if regularization == 'L2' else 0))

            # Entrenamiento del modelo
            for epoch in range(num_epochs):
                model.train()
                optimizer.zero_grad()
                outputs = model(X_train)
                loss = loss_function(outputs, y_train)
                
                # Aplicar regularización L1 si se seleccionó
                if regularization == 'L1':
                    l1_lambda = 1e-4
                    l1_norm = sum(p.abs().sum() for p in model.parameters())
                    loss += l1_lambda * l1_norm
                
                loss.backward()
                optimizer.step()

            # Validación del modelo
            model.eval()
            val_outputs = model(X_val)
            _, predicted = torch.max(val_outputs, 1)
            accuracy = accuracy_score(y_val, predicted)
            
            print(f'Loss function: {loss_function_name}, Regularization: {regularization}, Optimizer: {optimizer_name}, Validation Accuracy: {accuracy}')
            
            # Guardar la mejor combinación basada en la precisión de validación
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_combo = (loss_function_name, regularization, optimizer_name)

print(f'The best combination was Loss function: {best_combo[0]}, Regularization: {best_combo[1]}, Optimizer: {best_combo[2]} with a Validation Accuracy of {best_accuracy}')

Loss function: CrossEntropy, Regularization: L1, Optimizer: SGD, Validation Accuracy: 0.36666666666666664
Loss function: CrossEntropy, Regularization: L1, Optimizer: Adam, Validation Accuracy: 1.0
Loss function: CrossEntropy, Regularization: L1, Optimizer: RMSprop, Validation Accuracy: 0.8333333333333334
Loss function: CrossEntropy, Regularization: L2, Optimizer: SGD, Validation Accuracy: 0.36666666666666664
Loss function: CrossEntropy, Regularization: L2, Optimizer: Adam, Validation Accuracy: 0.9666666666666667
Loss function: CrossEntropy, Regularization: L2, Optimizer: RMSprop, Validation Accuracy: 0.8666666666666667
Loss function: CrossEntropy, Regularization: Dropout, Optimizer: SGD, Validation Accuracy: 0.7
Loss function: CrossEntropy, Regularization: Dropout, Optimizer: Adam, Validation Accuracy: 1.0
Loss function: CrossEntropy, Regularization: Dropout, Optimizer: RMSprop, Validation Accuracy: 0.9
Loss function: MSE, Regularization: L1, Optimizer: SGD, Validation Accuracy: 0.7
Lo